In [ ]:
# This notebook makes a lineage tree "from scratch" using source images and the low level API.

In [ ]:
# Uncompress the example label TIFF files
%ls

In [ ]:
%%bash

gunzip --keep tiffs.tar.gz
tar xfv tiffs.tar


In [ ]:
%ls tiffs/

In [ ]:
# images
%ls img/

In [ ]:
# destination directory for generated files
%mkdir -p destination

In [ ]:

from mouse_embryo_labeller import tools

# File system helper makes a new timestamp and nucleus collection
helper = tools.FileSystemHelper("./destination")


In [ ]:
# make processed timestamp data files
from mouse_embryo_labeller.tools import load_tiff_array
import pyklb

stride = 4 # stride for limiting image sizes
for (num, labelsfn, imagefn) in [
    (1, "tiffs/l1.tiff", "img/i1.klb"),
    (2, "tiffs/L2.tiff", "img/i2.klb"),
    (3, "tiffs/l3.tiff", "img/i3.klb"),
    (4, "tiffs/l4.tiff", "img/i4.klb"),
]:
    print("loading", (num, labelsfn, imagefn))
    img = pyklb.readfull(imagefn)
    labels = load_tiff_array(labelsfn)
    assert img.shape == labels.shape, "bad shapes " + repr((img.shape, labels.shape))
    # reduce the image sizes
    s_img = img[:, ::stride, ::stride]
    s_labels = labels[:, ::stride, ::stride]
    helper.add_timestamp(num, s_img, s_labels)

# get the nucleus collection and timestamp collection
tsc = helper.stored_timestamp_collection()
nc = helper.stored_nucleus_collection()

%ls destination

In [ ]:
# make some nucei (tracks)
n1 = nc.get_or_make_nucleus("n1")
n1.color[:] = (240, 128, 128)  # light coral
n2 = nc.get_or_make_nucleus("n2")
n2.color[:] = (255, 105, 180)  # hot pink
n3 = nc.get_or_make_nucleus("n3")
n3.color[:] = (255, 215, 0) # gold
n4 = nc.get_or_make_nucleus("n4")
n4.color[:] = (46, 139, 87) # sea green

# add some parent relationships
n3.reparent(n2.identifier)
n4.reparent(n2.identifier)

In [ ]:
# map some labels in timestamps to nuclei
ts1 = tsc.get_timestamp(1)
ts2 = tsc.get_timestamp(2)
ts3 = tsc.get_timestamp(3)
ts4 = tsc.get_timestamp(4)

ts1.assign_nucleus(6, n1)
ts2.assign_nucleus(4, n1)
ts1.assign_nucleus(1, n2)
ts2.assign_nucleus(1, n2)
ts3.assign_nucleus(7, n3)
ts3.assign_nucleus(6, n4)

tsc.store_all()
nc.save_json()

In [ ]:
from mouse_embryo_labeller import viz_controller
from mouse_embryo_labeller import tools

folder = "./destination"
nc = tools.get_example_nucleus_collection(folder)
tsc = tools.get_example_timestamp_collection(folder, nc)

v = viz_controller.VizController(folder, tsc, nc)

In [ ]:
v.make_widget()